# Context Engineering in LangChain & LangGraph

Context engineering is about building **dynamic systems** that provide the right information and tools in the right format so that an LLM can accomplish its task. Unlike prompt engineering (static, handcrafted strings), context engineering encompasses dynamic context construction using memory, state, and retrieval.

This notebook covers:

1. **Static Context** - Immutable data that doesn't change during execution
2. **Dynamic Context** - Mutable data that evolves as the application runs
3. **Runtime Context** - Data scoped to a single run or invocation
4. **Cross-Conversation Context** - Data that persists across multiple sessions

---

## Setup

First, let's install the required packages and configure our environment.

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langgraph

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

---

# Part 1: Static Context

**Static context** is immutable data that doesn't change during execution. Examples include:

- User metadata (name, preferences set at startup)
- Database connections
- Available tools
- Configuration settings
- System prompts

In LangChain, static context is passed to an agent at the start of a run via the `context` argument or embedded in the system prompt.

## 1.1 Static Context via System Prompts

The simplest form of static context - information baked into the system prompt:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# Static context embedded in system prompt
STATIC_SYSTEM_PROMPT = """You are a customer support agent for TechCorp.

Company Information (STATIC CONTEXT):
- Company: TechCorp Inc.
- Support Hours: Mon-Fri 9am-5pm EST
- Refund Policy: 30-day money-back guarantee
- Current Promotion: 20% off with code SAVE20

Use this information to help customers with their inquiries.
"""

model = ChatOpenAI(model="gpt-4o-mini")

messages = [
    SystemMessage(content=STATIC_SYSTEM_PROMPT),
    HumanMessage(content="What's your refund policy?")
]

response = model.invoke(messages)
print(response.content)

## 1.2 Static Context via Closure (Module-Level Config)

A simple pattern: define configuration once, and tools/functions access it via closure:

In [ ]:
# Static configuration - defined once, never changes during execution
PRODUCT_CATALOG = {
    "Widget Pro": {"price": 99.99, "in_stock": True},
    "Gadget Plus": {"price": 149.99, "in_stock": True},
    "Device Max": {"price": 299.99, "in_stock": False},
}

MAX_DISCOUNT_PERCENT = 25
SUPPORT_EMAIL = "support@techcorp.com"

from langchain_core.tools import tool

@tool
def check_product(product_name: str) -> str:
    """Check if a product exists and its availability."""
    # Tools access static context via closure
    if product_name in PRODUCT_CATALOG:
        product = PRODUCT_CATALOG[product_name]
        status = "in stock" if product["in_stock"] else "out of stock"
        return f"{product_name}: ${product['price']} ({status})"
    return f"Product '{product_name}' not found. Available: {list(PRODUCT_CATALOG.keys())}"

@tool
def calculate_price(product_name: str, discount_percent: int = 0) -> str:
    """Calculate final price with optional discount."""
    if product_name not in PRODUCT_CATALOG:
        return f"Product '{product_name}' not found."

    base_price = PRODUCT_CATALOG[product_name]["price"]
    # Static context (MAX_DISCOUNT_PERCENT) limits the discount
    actual_discount = min(discount_percent, MAX_DISCOUNT_PERCENT)
    final_price = base_price * (1 - actual_discount / 100)

    result = f"{product_name}: ${final_price:.2f}"
    if discount_percent > MAX_DISCOUNT_PERCENT:
        result += f" (discount capped at {MAX_DISCOUNT_PERCENT}%)"
    return result

# Test the tools - they use static context
print(check_product.invoke({"product_name": "Widget Pro"}))
print(check_product.invoke({"product_name": "Unknown Product"}))
print(calculate_price.invoke({"product_name": "Widget Pro", "discount_percent": 50}))

## 1.3 Static Context with ReAct Agent

Let's build a simple agent that uses these tools with static context:

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

# Create a ReAct agent with our tools
# The tools have access to static context (PRODUCT_CATALOG, MAX_DISCOUNT_PERCENT)
shop_assistant = create_react_agent(
    model=model,
    tools=[check_product, calculate_price],
    prompt="You are a helpful shop assistant. Use tools to help customers with product info and pricing."
)

In [ ]:
# The agent uses tools that access static context
result = shop_assistant.invoke({
    "messages": [{"role": "user", "content": "What's the price of Widget Pro with a 20% discount?"}]
})

for msg in result["messages"]:
    if hasattr(msg, 'pretty_print'):
        msg.pretty_print()
    else:
        print(f"{msg['role']}: {msg['content']}")

In [ ]:
# Another query - notice the static context (product catalog) is always available
result = shop_assistant.invoke({
    "messages": [{"role": "user", "content": "Is Device Max available? And what products do you have?"}]
})

print(result["messages"][-1].content)

---

# Part 2: Dynamic Context

**Dynamic context** is mutable data that evolves as the application runs. Examples include:

- Conversation history (messages accumulating over time)
- Intermediate results from tool calls
- Scratchpad notes the agent writes to itself
- Computed values that change based on user actions

In LangGraph, dynamic context is managed through the **state object**.

## 2.1 Message History as Dynamic Context

The most common form of dynamic context - the conversation grows with each interaction:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model = ChatOpenAI(model="gpt-4o-mini")

# Dynamic context: message history that grows over time
conversation_history = [
    SystemMessage(content="You are a helpful math tutor. Be concise.")
]

def chat(user_input: str):
    """Add user message, get response, update history."""
    # Add user message to dynamic context
    conversation_history.append(HumanMessage(content=user_input))
    
    # Model sees full context
    response = model.invoke(conversation_history)
    
    # Add AI response to dynamic context
    conversation_history.append(response)
    
    return response.content

# Watch the context grow dynamically
print("Turn 1:", chat("What is 5 + 3?"))
print(f"\nContext size: {len(conversation_history)} messages")

print("\nTurn 2:", chat("Multiply that by 2"))
print(f"\nContext size: {len(conversation_history)} messages")

print("\nTurn 3:", chat("What was my first question?"))
print(f"\nContext size: {len(conversation_history)} messages")

## 2.2 Scratchpad: Agent's Working Memory

A scratchpad allows the agent to write notes to itself during execution - dynamic context that evolves as the agent works:

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated, List
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

class ScratchpadState(TypedDict):
    """State with a scratchpad for dynamic notes."""
    messages: Annotated[list, add_messages]
    scratchpad: List[str]  # Dynamic context: notes the agent writes
    step_count: int        # Dynamic context: tracks progress

model = ChatOpenAI(model="gpt-4o-mini")

def analyze_and_note(state: ScratchpadState):
    """Agent analyzes input and writes to scratchpad."""
    user_msg = state["messages"][-1].content
    current_notes = state.get("scratchpad", [])
    step = state.get("step_count", 0) + 1
    
    # Build prompt with current scratchpad context
    scratchpad_context = "\n".join(current_notes) if current_notes else "(empty)"
    
    prompt = f"""Analyze this request and write a brief note about what you learned.

Current scratchpad:
{scratchpad_context}

User request: {user_msg}

Respond with:
1. A note to add to your scratchpad (start with "NOTE:")
2. Your response to the user (start with "RESPONSE:")"""
    
    response = model.invoke([HumanMessage(content=prompt)])
    content = response.content
    
    # Extract note and response
    note = ""
    reply = content
    if "NOTE:" in content and "RESPONSE:" in content:
        parts = content.split("RESPONSE:")
        note = parts[0].replace("NOTE:", "").strip()
        reply = parts[1].strip()
    
    # Update dynamic context
    new_notes = current_notes + [f"Step {step}: {note}"] if note else current_notes
    
    return {
        "messages": [response],
        "scratchpad": new_notes,
        "step_count": step
    }

# Build graph
builder = StateGraph(ScratchpadState)
builder.add_node("analyze", analyze_and_note)
builder.add_edge(START, "analyze")
builder.add_edge("analyze", END)

scratchpad_graph = builder.compile()

In [ ]:
# Run multiple interactions, watching scratchpad grow
state = {
    "messages": [HumanMessage(content="I need help planning a trip to Japan")],
    "scratchpad": [],
    "step_count": 0
}

result = scratchpad_graph.invoke(state)
print("After turn 1:")
print(f"Scratchpad: {result['scratchpad']}")
print(f"Step count: {result['step_count']}")

# Continue conversation
result["messages"].append(HumanMessage(content="I'm interested in traditional culture"))
result = scratchpad_graph.invoke(result)
print("\nAfter turn 2:")
print(f"Scratchpad: {result['scratchpad']}")
print(f"Step count: {result['step_count']}")

## 2.3 Tool Results as Dynamic Context

When agents call tools, the results become part of the dynamic context that influences subsequent decisions:

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Dynamic context: a shopping cart that changes based on tool calls
shopping_cart = []

@tool
def add_to_cart(item: str, quantity: int = 1) -> str:
    """Add an item to the shopping cart."""
    for _ in range(quantity):
        shopping_cart.append(item)
    return f"Added {quantity}x {item} to cart. Cart now has {len(shopping_cart)} items."

@tool
def view_cart() -> str:
    """View the current shopping cart contents."""
    if not shopping_cart:
        return "Cart is empty."
    from collections import Counter
    counts = Counter(shopping_cart)
    items = [f"{item}: {count}" for item, count in counts.items()]
    return f"Cart contents: {', '.join(items)}"

@tool
def clear_cart() -> str:
    """Clear all items from the cart."""
    shopping_cart.clear()
    return "Cart cleared!"

model = ChatOpenAI(model="gpt-4o-mini")

shop_agent = create_react_agent(
    model=model,
    tools=[add_to_cart, view_cart, clear_cart],
    prompt="You are a shopping assistant. Help users manage their cart using the available tools."
)

In [ ]:
# Watch the dynamic context (cart) evolve
clear_cart.invoke({})  # Start fresh

print("=== Shopping Session ===")
for request in [
    "Add 2 apples to my cart",
    "Also add a banana",
    "What's in my cart?"
]:
    print(f"\nUser: {request}")
    result = shop_agent.invoke({"messages": [{"role": "user", "content": request}]})
    print(f"Agent: {result['messages'][-1].content}")

---

# Part 3: Runtime Context

**Runtime context** is data scoped to a single run or invocation. It exists only for the duration of one agent execution and is discarded afterward.

Key characteristics:
- Isolated to a single `invoke()` or `stream()` call
- Not persisted between runs
- Often includes request-specific data like user input, session tokens, or temporary calculations

## 3.1 Configurable Runtime Parameters

Use `RunnableConfig` to pass runtime-specific parameters:

In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain_openai import ChatOpenAI
from typing import Annotated
from langchain_core.tools import tool, InjectedToolArg

@tool
def get_personalized_greeting(
    time_of_day: str,
    config: Annotated[RunnableConfig, InjectedToolArg],
) -> str:
    """Generate a personalized greeting based on runtime context."""
    # Access runtime-specific configuration
    user_id = config.get("configurable", {}).get("user_id", "guest")
    language = config.get("configurable", {}).get("language", "en")
    
    greetings = {
        "en": {"morning": "Good morning", "afternoon": "Good afternoon", "evening": "Good evening"},
        "es": {"morning": "Buenos días", "afternoon": "Buenas tardes", "evening": "Buenas noches"},
        "ja": {"morning": "おはようございます", "afternoon": "こんにちは", "evening": "こんばんは"},
    }
    
    greeting = greetings.get(language, greetings["en"]).get(time_of_day, "Hello")
    return f"{greeting}, User {user_id}!"

# Test with different runtime contexts
print("Runtime context 1 (English):")
print(get_personalized_greeting.invoke(
    {"time_of_day": "morning"},
    config={"configurable": {"user_id": "123", "language": "en"}}
))

print("\nRuntime context 2 (Japanese):")
print(get_personalized_greeting.invoke(
    {"time_of_day": "morning"},
    config={"configurable": {"user_id": "456", "language": "ja"}}
))

## 3.2 Runtime State in LangGraph

LangGraph's state object acts as runtime context - it exists only during graph execution:

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Optional
from langchain_openai import ChatOpenAI

class CalculationState(TypedDict):
    """Runtime state - exists only during this execution."""
    numbers: list[float]
    operation: str
    intermediate_result: Optional[float]
    final_result: Optional[float]
    error: Optional[str]

def validate_input(state: CalculationState):
    """Validate input and set error if needed."""
    if not state["numbers"]:
        return {"error": "No numbers provided"}
    if state["operation"] not in ["sum", "product", "average"]:
        return {"error": f"Unknown operation: {state['operation']}"}
    return {"error": None}

def calculate(state: CalculationState):
    """Perform calculation and store intermediate result."""
    if state.get("error"):
        return {}  # Skip if validation failed
    
    numbers = state["numbers"]
    op = state["operation"]
    
    if op == "sum":
        result = sum(numbers)
    elif op == "product":
        result = 1
        for n in numbers:
            result *= n
    elif op == "average":
        result = sum(numbers) / len(numbers)
    
    return {"intermediate_result": result}

def finalize(state: CalculationState):
    """Round and finalize the result."""
    if state.get("error"):
        return {"final_result": None}
    return {"final_result": round(state["intermediate_result"], 2)}

# Build the graph
builder = StateGraph(CalculationState)
builder.add_node("validate", validate_input)
builder.add_node("calculate", calculate)
builder.add_node("finalize", finalize)

builder.add_edge(START, "validate")
builder.add_edge("validate", "calculate")
builder.add_edge("calculate", "finalize")
builder.add_edge("finalize", END)

calc_graph = builder.compile()

In [ ]:
# Each invocation has its own isolated runtime context
result1 = calc_graph.invoke({
    "numbers": [1, 2, 3, 4, 5],
    "operation": "average",
    "intermediate_result": None,
    "final_result": None,
    "error": None
})
print(f"Run 1 - Average of [1,2,3,4,5]: {result1['final_result']}")

result2 = calc_graph.invoke({
    "numbers": [2, 3, 4],
    "operation": "product",
    "intermediate_result": None,
    "final_result": None,
    "error": None
})
print(f"Run 2 - Product of [2,3,4]: {result2['final_result']}")

# Each run is completely isolated - no state leaks between them

## 3.3 Thread-Scoped Runtime Context with Checkpointing

Using `MemorySaver`, you can maintain runtime context within a single "thread" or session:

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI

class ChatState(TypedDict):
    messages: Annotated[list, add_messages]

model = ChatOpenAI(model="gpt-4o-mini")

def chat_node(state: ChatState):
    response = model.invoke(state["messages"])
    return {"messages": [response]}

builder = StateGraph(ChatState)
builder.add_node("chat", chat_node)
builder.add_edge(START, "chat")
builder.add_edge("chat", END)

# Add checkpointer for runtime persistence within a thread
checkpointer = MemorySaver()
chat_graph = builder.compile(checkpointer=checkpointer)

In [ ]:
# Thread 1: Maintain context within this runtime session
config1 = {"configurable": {"thread_id": "session_abc"}}

result = chat_graph.invoke(
    {"messages": [{"role": "user", "content": "My name is Alice. Remember this!"}]},
    config1
)
print("Thread 1, Turn 1:", result["messages"][-1].content)

# Continue same thread - context persists
result = chat_graph.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    config1
)
print("Thread 1, Turn 2:", result["messages"][-1].content)

In [ ]:
# Thread 2: Completely separate runtime context
config2 = {"configurable": {"thread_id": "session_xyz"}}

result = chat_graph.invoke(
    {"messages": [{"role": "user", "content": "What's my name?"}]},
    config2
)
print("Thread 2 (new session):", result["messages"][-1].content)
# This thread doesn't know Alice - runtime contexts are isolated!

---

# Part 4: Cross-Conversation Context

**Cross-conversation context** persists across multiple conversations or sessions. This is long-term memory that survives beyond a single thread.

Examples:
- User preferences learned over time
- Historical interactions
- Facts about the user (name, location, interests)
- Accumulated knowledge from past sessions

In LangGraph, this is managed through the **InMemoryStore** or persistent storage backends.

## 4.1 InMemoryStore Basics

The `InMemoryStore` provides namespace-based storage for long-term memory:

In [ ]:
from langgraph.store.memory import InMemoryStore
import uuid

# Create the store
store = InMemoryStore()

# Store information in a user-specific namespace
user_id = "user_123"
namespace = (user_id, "preferences")  # Tuple creates hierarchical namespace

# Store some preferences
store.put(namespace, "food", {"preference": "vegetarian", "favorite_cuisine": "Italian"})
store.put(namespace, "communication", {"preferred_time": "morning", "style": "formal"})

# Retrieve all items in namespace
items = store.search(namespace)
print("Stored preferences:")
for item in items:
    print(f"  {item.key}: {item.value}")

In [ ]:
# Retrieve specific item
food_pref = store.get(namespace, "food")
print(f"Food preference: {food_pref.value}")

# Update an item (upsert)
store.put(namespace, "food", {"preference": "vegan", "favorite_cuisine": "Japanese"})
updated = store.get(namespace, "food")
print(f"Updated food preference: {updated.value}")

## 4.2 Cross-Conversation Agent with Long-Term Memory

Let's build an agent that remembers facts about users across different conversation threads:

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.store.base import BaseStore
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langchain_core.runnables import RunnableConfig
from typing import TypedDict, Annotated
import json

class MemoryState(TypedDict):
    messages: Annotated[list, add_messages]

model = ChatOpenAI(model="gpt-4o-mini")

def chat_with_memory(state: MemoryState, config: RunnableConfig, *, store: BaseStore):
    """Chat node that reads and writes to long-term memory."""
    user_id = config["configurable"].get("user_id", "anonymous")
    namespace = (user_id, "facts")
    
    # Retrieve existing memories
    memories = store.search(namespace)
    memory_text = "\n".join([f"- {m.key}: {m.value}" for m in memories])
    
    system_prompt = f"""You are a helpful assistant with long-term memory.
    
Known facts about this user:
{memory_text if memory_text else "(No facts stored yet)"}

IMPORTANT: When the user shares new personal information (name, preferences, interests, etc.),
extract it and include it in your response in this format:
[REMEMBER: key=value]

For example: [REMEMBER: name=Alice] or [REMEMBER: favorite_color=blue]"""
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    
    # Parse and store any new facts
    content = response.content
    import re
    facts = re.findall(r'\[REMEMBER: (\w+)=([^\]]+)\]', content)
    for key, value in facts:
        store.put(namespace, key, value)
        content = content.replace(f"[REMEMBER: {key}={value}]", "")
    
    response.content = content.strip()
    return {"messages": [response]}

# Build the graph with both checkpointer and store
builder = StateGraph(MemoryState)
builder.add_node("chat", chat_with_memory)
builder.add_edge(START, "chat")
builder.add_edge("chat", END)

checkpointer = MemorySaver()  # For runtime/thread context
memory_store = InMemoryStore()  # For cross-conversation context

memory_agent = builder.compile(checkpointer=checkpointer, store=memory_store)

In [ ]:
# Conversation 1: User shares information
config = {"configurable": {"thread_id": "conv_1", "user_id": "user_alice"}}

result = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "Hi! My name is Alice and I love hiking."}]},
    config
)
print("Conversation 1:")
print(f"User: Hi! My name is Alice and I love hiking.")
print(f"Agent: {result['messages'][-1].content}")

In [ ]:
# Check what was stored
stored_facts = memory_store.search(("user_alice", "facts"))
print("\nStored cross-conversation facts:")
for fact in stored_facts:
    print(f"  {fact.key}: {fact.value}")

In [ ]:
# Conversation 2: NEW thread, but same user - facts persist!
config = {"configurable": {"thread_id": "conv_2", "user_id": "user_alice"}}

result = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "What do you remember about me?"}]},
    config
)
print("Conversation 2 (NEW thread, same user):")
print(f"User: What do you remember about me?")
print(f"Agent: {result['messages'][-1].content}")

In [ ]:
# Conversation 3: Different user - no cross-contamination
config = {"configurable": {"thread_id": "conv_3", "user_id": "user_bob"}}

result = memory_agent.invoke(
    {"messages": [{"role": "user", "content": "What do you know about me?"}]},
    config
)
print("Conversation 3 (DIFFERENT user - Bob):")
print(f"User: What do you know about me?")
print(f"Agent: {result['messages'][-1].content}")

## 4.3 Simplified Long-Term Memory Pattern

Here's a simpler pattern using module-level storage that persists across function calls:

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Simple in-memory user database (cross-conversation persistence)
USER_MEMORIES = {}

@tool
def remember_fact(user_id: str, fact_type: str, fact_value: str) -> str:
    """Store a fact about a user for future conversations.
    
    Args:
        user_id: The user's identifier
        fact_type: Category of the fact (e.g., 'name', 'preference', 'hobby')
        fact_value: The actual fact to remember
    """
    if user_id not in USER_MEMORIES:
        USER_MEMORIES[user_id] = {}
    USER_MEMORIES[user_id][fact_type] = fact_value
    return f"Remembered: {user_id}'s {fact_type} is '{fact_value}'"

@tool
def recall_facts(user_id: str) -> str:
    """Recall all stored facts about a user.
    
    Args:
        user_id: The user's identifier
    """
    if user_id not in USER_MEMORIES or not USER_MEMORIES[user_id]:
        return f"No memories stored for user '{user_id}'"
    
    facts = [f"- {k}: {v}" for k, v in USER_MEMORIES[user_id].items()]
    return f"Memories for {user_id}:\n" + "\n".join(facts)

model = ChatOpenAI(model="gpt-4o-mini")

memory_agent = create_react_agent(
    model=model,
    tools=[remember_fact, recall_facts],
    prompt="""You are a helpful assistant with long-term memory.

When users share personal information, use remember_fact to store it.
When users ask what you know about them, use recall_facts.
Always use the user_id provided in the conversation."""
)

In [ ]:
# Session 1: User shares information
print("=== Session 1: Charlie introduces himself ===")
result = memory_agent.invoke({
    "messages": [{"role": "user", "content": "Hi! I'm user charlie_123. My name is Charlie, I'm a software engineer, and I love pizza."}]
})
print(f"Agent: {result['messages'][-1].content}")

# Check what was stored
print(f"\nStored memories: {USER_MEMORIES}")

In [ ]:
# Session 2: Completely new conversation - but memories persist!
print("=== Session 2: New conversation, same user ===")
result = memory_agent.invoke({
    "messages": [{"role": "user", "content": "I'm charlie_123. What do you remember about me?"}]
})
print(f"Agent: {result['messages'][-1].content}")

# Session 3: Different user - isolated memories
print("\n=== Session 3: Different user (bob_456) ===")
result = memory_agent.invoke({
    "messages": [{"role": "user", "content": "I'm bob_456. What do you know about me?"}]
})
print(f"Agent: {result['messages'][-1].content}")

---

# Summary: Context Types Comparison

| Context Type | Mutability | Lifetime | LangChain Mechanism | Example |
|-------------|------------|----------|--------------------|---------|
| **Static** | Immutable | Application lifetime | System prompts, `context` arg | User tier, API keys, tools |
| **Dynamic** | Mutable | Within a run | StateGraph state | Message history, scratchpad |
| **Runtime** | Varies | Single invocation | `RunnableConfig`, thread_id | Request-specific params |
| **Cross-Conversation** | Persistent | Multiple sessions | `InMemoryStore` | User preferences, facts |

## Key Takeaways

1. **Think of context as RAM** - Deliberately curate what enters the LLM's context window at each step

2. **Four strategies for context management**:
   - **Write**: Save context outside the window (scratchpad, store)
   - **Select**: Pull relevant context in (search, filtering)
   - **Compress**: Summarize to reduce tokens
   - **Isolate**: Separate contexts across agents/fields

3. **Choose the right tool**:
   - `MemorySaver` for thread/session persistence (runtime context)
   - `InMemoryStore` for cross-conversation persistence (long-term memory)
   - State fields for dynamic in-run context
   - System prompts/dataclasses for static context

---

**Resources:**
- [LangChain Context Documentation](https://docs.langchain.com/oss/python/concepts/context)
- [Context Engineering Blog Post](https://blog.langchain.com/context-engineering-for-agents/)
- [LangGraph Memory Guide](https://docs.langchain.com/oss/python/langgraph/memory)